# Simplifying the Yelp Reviews Dataset

The dataset given to us by Kaggle contains an extensive review table that lists every single review on Yelp for every business in the dataset; over 5 million of them. We are not interested in a lot of the information in this table.

This notebook creates two new tables:

- 1) Uses the reviews to calculate the start and end date of a businesses lifespan on Yelp.
- 2) Condenses 5 million reviews to 174,000 lines where each one represents a business, and a column details the date and rating of every review n tuple form for easy access.

In [1]:
import os
import sys
import pandas as pd

module_path = os.path.abspath(os.path.join('../../data/..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
%matplotlib inline

We will be adjusting the yelp review dataset, and using some information from the business table.

In [2]:
df = pd.read_csv("../../data/raw/yelp_review.csv")
bus = pd.read_csv("../../data/raw/yelp_business.csv")

The review dataset gives us a lot of info that we don't need; user id, the text of the review, and how funny/useful other reviewers found it do not matter for our analysis.

In [3]:
df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,2016-05-28,Super simple place but amazing nonetheless. It...,0,0,0
1,n6QzIUObkYshz4dz2QRJTw,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,2016-05-28,Small unassuming place that changes their menu...,0,0,0
2,MV3CcKScW05u5LVfF6ok0g,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5,2016-05-28,Lester's is located in a beautiful neighborhoo...,0,0,0
3,IXvOzsEMYtiJI0CARmj77Q,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4,2016-05-28,Love coming here. Yes the place always needs t...,0,0,0
4,L_9BTb55X0GDtThi6GlZ6w,bv2nCi5Qv5vroFiqKGopiw,s2I_Ni76bjJNK9yG60iD-Q,4,2016-05-28,Had their chocolate almond croissant and it wa...,0,0,0


## 1) Make a table detailing the lifespan of each business

Make a dataframe that details the first and last review of each business

In [4]:
#Filter the review dataset to just give the info we need and format dates correctly
rev = df.filter(['review_id', 'business_id', 'stars', 'date'])
rev['date'] =  pd.to_datetime(rev['date'], format='%Y-%m-%d')

#Assemble all the reviews of one business into a group.
groups = rev.groupby('business_id')
newRev = pd.DataFrame(groups.max()['date']) #get the latest date
newRev2 = pd.DataFrame(groups.min()['date']) #get the earliest date

#Make a datframe with the earliest and latest review of each business
dates = pd.concat([newRev, newRev2], axis=1)
dates.columns = ['lastReview', 'firstReview']

dates.head()

,lastReview,firstReview
business_id,,
--6MefnULPED_I942VcFNA,2017-12-04,2008-08-07
--7zmmkVg-IMGaXbuVd0SQ,2017-11-12,2014-12-30
--8LPVSo5i0Oo61X01sV9A,2017-06-28,2014-05-15
--9QQLMTbFzLJ_oT-ON3Xw,2017-05-02,2013-05-15
--9e1ONYQuAa-CB_Rrw7Tw,2017-12-11,2005-04-27


Calculate the lifespan of each business in days and convert this value from datetime to an int

In [5]:
dates['lifespan'] = dates.lastReview - dates.firstReview #lifespan if latest review in days, minus first review in days

def convert(a):
    b = a.total_seconds() #convert number to total seconds
    totSec = b/86400 #divide the seconds to give the amount of days
    totSec = int(totSec) #convert to int
    return totSec

dates['lifespan'] = dates.apply(lambda row: convert(row['lifespan']), axis=1) #convert all of the days to ints
dates.head()

,lastReview,firstReview,lifespan
business_id,,,
--6MefnULPED_I942VcFNA,2017-12-04,2008-08-07,3406
--7zmmkVg-IMGaXbuVd0SQ,2017-11-12,2014-12-30,1048
--8LPVSo5i0Oo61X01sV9A,2017-06-28,2014-05-15,1140
--9QQLMTbFzLJ_oT-ON3Xw,2017-05-02,2013-05-15,1448
--9e1ONYQuAa-CB_Rrw7Tw,2017-12-11,2005-04-27,4611


Combine the business info and dates info we just made

In [6]:
biz = bus.filter(['business_id', 'name', 'stars', 'city', 'state', 'postal_code', 'review_count', 'is_open'])
biz = biz.set_index('business_id')
revInfo = pd.concat([biz, dates], axis=1)

revInfo.head()

,name,stars,city,state,postal_code,review_count,is_open,lastReview,firstReview,lifespan
--6MefnULPED_I942VcFNA,"""John's Chinese BBQ Restaurant""",3.0,Richmond Hill,ON,L4B 3P7,37,1,2017-12-04,2008-08-07,3406
--7zmmkVg-IMGaXbuVd0SQ,"""Primal Brewery""",4.0,Huntersville,NC,28078,47,1,2017-11-12,2014-12-30,1048
--8LPVSo5i0Oo61X01sV9A,"""Valley Bone and Joint Specialists""",4.5,Gilbert,AZ,85234,3,1,2017-06-28,2014-05-15,1140
--9QQLMTbFzLJ_oT-ON3Xw,"""Great Clips""",3.5,Tempe,AZ,85283,11,1,2017-05-02,2013-05-15,1448
--9e1ONYQuAa-CB_Rrw7Tw,"""Delmonico Steakhouse""",4.0,Las Vegas,NV,89109,1451,1,2017-12-11,2005-04-27,4611


Save the lifespan table to a pickle for use in RQ1.1

In [7]:
revInfo.to_pickle('../../data/prep/reviewInfo.pkl')

## 2) Making a simpler table that stores every review and rating for each business

Make a column 'revrate' that combines the review date and rating into a tuple

In [8]:
dateStudy = df.filter(['review_id', 'business_id', 'stars', 'date']) #only take thte columns we need
review_list = list(zip(dateStudy.date, dateStudy.stars)) #convert date and time of each review to a tuple
dateStudy['revrate'] = pd.Series(review_list) #make this a column in the dataset
dateStudy.head()

,review_id,business_id,stars,date,revrate
0,vkVSCC7xljjrAI4UGfnKEQ,AEx2SYEUJmTxVVB18LlCwA,5,2016-05-28,"(2016-05-28, 5)"
1,n6QzIUObkYshz4dz2QRJTw,VR6GpWIda3SfvPC-lg9H3w,5,2016-05-28,"(2016-05-28, 5)"
2,MV3CcKScW05u5LVfF6ok0g,CKC0-MOWMqoeWf6s-szl8g,5,2016-05-28,"(2016-05-28, 5)"
3,IXvOzsEMYtiJI0CARmj77Q,ACFtxLv8pGrrxMm6EgjreA,4,2016-05-28,"(2016-05-28, 4)"
4,L_9BTb55X0GDtThi6GlZ6w,s2I_Ni76bjJNK9yG60iD-Q,4,2016-05-28,"(2016-05-28, 4)"


Group the entire table of all reviews by business and put all the tuples into a list.

What we have now is the index where each one is an individual business, and a column that is a list of all its review dates and ratings.

In [9]:
dateLists = dateStudy.groupby('business_id')['revrate'].apply(list)
reviewsTemplate = pd.DataFrame(dateLists)  # Make a dataframe of business id, all review info
reviewsTemplate.head()

,revrate
business_id,
--6MefnULPED_I942VcFNA,"[(2017-08-17, 4), (2017-05-31, 3), (2016-10-23..."
--7zmmkVg-IMGaXbuVd0SQ,"[(2015-11-30, 4), (2017-08-16, 4), (2016-08-18..."
--8LPVSo5i0Oo61X01sV9A,"[(2016-07-29, 3), (2017-06-28, 5), (2014-05-15..."
--9QQLMTbFzLJ_oT-ON3Xw,"[(2016-04-02, 2), (2013-05-15, 4), (2017-05-02..."
--9e1ONYQuAa-CB_Rrw7Tw,"[(2011-08-24, 5), (2016-03-31, 5), (2015-06-29..."


Use info from the business table to record if the business is open or closed.

In [10]:
openClose = bus.filter(['business_id', 'is_open']) #get the "is_open" info from the original business dataframe
openClose = openClose.set_index('business_id') #set the index to business id

#Combine new review info and is_open info into a dataframe
reviews = pd.concat([reviewsTemplate, openClose], axis=1) 
reviews.head()

,revrate,is_open
--6MefnULPED_I942VcFNA,"[(2017-08-17, 4), (2017-05-31, 3), (2016-10-23...",1
--7zmmkVg-IMGaXbuVd0SQ,"[(2015-11-30, 4), (2017-08-16, 4), (2016-08-18...",1
--8LPVSo5i0Oo61X01sV9A,"[(2016-07-29, 3), (2017-06-28, 5), (2014-05-15...",1
--9QQLMTbFzLJ_oT-ON3Xw,"[(2016-04-02, 2), (2013-05-15, 4), (2017-05-02...",1
--9e1ONYQuAa-CB_Rrw7Tw,"[(2011-08-24, 5), (2016-03-31, 5), (2015-06-29...",1


Save this to pickle for use in RQ1.2

In [11]:
reviews.to_pickle('../../data/prep/everyReview.pkl')